In [1]:
import numpy as np
import pandas as pd
from typing import List
from firecrawl import FirecrawlApp
from pydantic import BaseModel, Field
from datetime import datetime

app = FirecrawlApp(api_key='fc-ec0d4c7dd15a47eb8050aff32f39cf4a')

In [2]:
# Model for a single product
class Mobile(BaseModel):
    product_name: str
    list_price: float = Field(None)
    deal_price: float = Field(None)
    delivery_by: str = Field(None)

# Model for the data containing multiple mobiles
class Data(BaseModel):
    mobiles: List[Mobile]

In [3]:
df = app.extract(
    ['https://www.sangeethamobiles.com/product-list/category-smartphones-308?sort_by=popularity'], 
    {'prompt': "Fetch information about mobile phones listed on this page: name of the product, its price, the deal price and delivery-by date",
     'schema': Data.model_json_schema() })

In [46]:
df_mobiles = pd.DataFrame(df['data']['mobiles'])
df_mobiles['price'] = np.minimum(df_mobiles['deal_price'], df_mobiles['list_price'])
df_mobiles['isAvailable'] = df_mobiles['delivery_by'].apply(lambda x: False if x == '' else True)
df_mobiles = df_mobiles.sort_values(by='price', ascending=False, ignore_index=True)
df_mobiles['record_date'] = datetime.now().strftime('%Y-%m-%d')
df_mobiles = df_mobiles[['record_date', 'product_name', 'price', 'isAvailable']]

display(df_mobiles)

,record_date,product_name,price,isAvailable
0,2025-03-25,Apple iPhone 11 Pro Max Silver 512GB,150800,False
1,2025-03-25,Vivo X80 Pro Cosmic Black 12GB 256GB,79999,True
2,2025-03-25,OnePlus 10 Pro Volcanic Black 12GB 256GB,71999,False
3,2025-03-25,Realme GT 2 Pro Paper White 12GB 256GB,48999,True
4,2025-03-25,OnePlus 9R Carbon Black 12GB 256GB,43999,False
5,2025-03-25,OnePlus 9 Arctic Skey 12GB 256GB,42999,False
6,2025-03-25,OnePlus 9R Carbon Black 8GB 128GB,39999,False
7,2025-03-25,OnePlus 10R Forest Green 12GB 256GB,35999,False
8,2025-03-25,Vivo V23 Pro 5G Stardust Black 12GB 256GB,34915,True
9,2025-03-25,Samsung Galaxy A52s 5G Light Violet 8GB 128GB,33999,False


In [4]:
df_mobiles = pd.DataFrame(df['data']['mobiles'])
df_mobiles['price'] = np.minimum(df_mobiles['deal_price'], df_mobiles['list_price'])
df_mobiles['isAvailable'] = df_mobiles['delivery_by'].apply(lambda x: False if x == '' else True)
df_mobiles = df_mobiles.sort_values(by='price', ascending=False, ignore_index=True)
df_mobiles['record_date'] = datetime.now().strftime('%Y-%m-%d')
df_mobiles = df_mobiles[['record_date', 'product_name', 'price', 'isAvailable']]

display(df_mobiles)

,record_date,product_name,price,isAvailable
0,2025-07-25,Apple iPhone 11 Pro Max Silver 512GB,94700,False
1,2025-07-25,Vivo X80 Pro Cosmic Black 12GB 256GB,44700,True
2,2025-07-25,Samsung S21 FE 5G Graphite 8GB 128GB,30200,True
3,2025-07-25,Realme GT 2 Pro Paper White 12GB 256GB,27400,True
4,2025-07-25,Vivo V23 Pro 5G Stardust Black 12GB 256GB,21300,True
5,2025-07-25,Oppo Reno7 Pro Star Light Black 12GB 256GB,18360,True
6,2025-07-25,Vivo Y75 5G Glowing Galaxy 8GB 128GB,14667,True
7,2025-07-25,Samsung Galaxy A22 5G Gray 8GB 128GB,14406,True
8,2025-07-25,Vivo T1 Pro Turbo Cyan 6GB 128GB,14100,True
9,2025-07-25,Oppo F21 Pro Cosmic Black 8GB 128GB,11499,True


In [6]:
avg_top10 = df_mobiles[df_mobiles['isAvailable'] == True]['price'].head(5).mean().astype(int)
avg_bot10 = df_mobiles[df_mobiles['isAvailable'] == True]['price'].tail(5).mean().astype(int)

print(f"Averge price of Top 5 most popular mobiles: ₹{avg_top10:,.0f}")

print(f"Averge price of Bottom 5 most popular mobiles: ₹{avg_bot10:,.0f}")

Averge price of Top 5 most popular mobiles: ₹28,392
Averge price of Bottom 5 most popular mobiles: ₹11,353
